In [27]:
import torch
import pandas as pd
from torch_geometric.datasets import MoleculeNet
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from torch.utils.data import Subset
from torch_geometric.utils.convert import to_networkx
from networkx import all_pairs_shortest_path
import numpy as np
from torch_geometric.utils.smiles import from_smiles

In [28]:
path = "/Users/rhysmcalister/Documents/GitHub/pyg_chemprop/data/CHNOPS2743-HARMONIC-processed-scaled.csv"
df = pd.read_csv(path)



In [32]:
df[labels].iloc[0]

400     6.508367e-04
402     8.458295e-04
404     7.231641e-04
406     5.875207e-04
408     6.762332e-04
            ...     
3992    1.024499e-08
3994    8.325449e-09
3996    1.259408e-08
3998    1.129142e-08
4000    1.941509e-08
Name: 0, Length: 1801, dtype: float64

In [90]:
input_data = from_smiles(df['SMILES'][0])
input_data
labels = np.arange(400,4002,2).astype(str)
dataset = []

for i in range(len(df)):
    input_data = from_smiles(df['SMILES'][i])

    dataset.append(input_data)


for x in dataset:
    x.y = torch.tensor([df[labels].iloc[i].values])
    x.x = x.x.to(torch.float32)
    x.edge_index = x.edge_index.to(torch.float32) 
    x.edge_attr = x.edge_attr.to(torch.float32)
# create a DataLoader
loader = DataLoader(dataset, batch_size=32, shuffle=True)



In [94]:
from torch import nn
import torch_geometric.nn as tgnn
from graphormer.model import Graphormer


model = Graphormer(
    num_layers=3,
    input_node_dim=input_data.num_node_features,
    node_dim=128,
    input_edge_dim=input_data.num_edge_features,
    edge_dim=128,
    output_dim=1801,
    n_heads=4,
    max_in_degree=5,
    max_out_degree=5,
    max_path_distance=5,
)

In [95]:
model

Graphormer(
  (node_in_lin): Linear(in_features=9, out_features=128, bias=True)
  (edge_in_lin): Linear(in_features=3, out_features=128, bias=True)
  (centrality_encoding): CentralityEncoding()
  (spatial_encoding): SpatialEncoding()
  (layers): ModuleList(
    (0-2): 3 x GraphormerEncoderLayer(
      (attention): GraphormerMultiHeadAttention(
        (heads): ModuleList(
          (0-3): 4 x GraphormerAttentionHead(
            (edge_encoding): EdgeEncoding()
            (q): Linear(in_features=128, out_features=128, bias=True)
            (k): Linear(in_features=128, out_features=128, bias=True)
            (v): Linear(in_features=128, out_features=128, bias=True)
          )
        )
        (linear): Linear(in_features=512, out_features=128, bias=True)
      )
      (ln_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (ln_2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (ff): Linear(in_features=128, out_features=128, bias=True)
    )
  )
  (node_out_l

In [96]:
from sklearn.model_selection import train_test_split

test_ids, train_ids = train_test_split([i for i in range(len(dataset))], test_size=0.8, random_state=42)
train_loader = DataLoader(Subset(dataset, train_ids), batch_size=64)
test_loader = DataLoader(Subset(dataset, test_ids), batch_size=64)

In [98]:
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
# loss_functin = nn.L1Loss(reduction="sum")
# loss_functin = nn.MSELoss(reduction="sum")

In [99]:
from tqdm import tqdm
from torch_geometric.nn.pool import global_mean_pool

# DEVICE = "cuda"
DEVICE = "cpu"

model.to(DEVICE)
for epoch in range(3):
    model.train()
    batch_loss = 0.0
    for batch in tqdm(train_loader):
        batch.to(DEVICE)
        y = batch.y
        optimizer.zero_grad()
        output = global_mean_pool(model(batch), batch.batch)
        loss = loss_functin(output, y)
        batch_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
    print("TRAIN_LOSS", batch_loss / len(train_ids))

    model.eval()
    batch_loss = 0.0
    for batch in tqdm(test_loader):
        batch.to(DEVICE)
        y = batch.y
        with torch.no_grad():
            output = global_mean_pool(model(batch), batch.batch)
            loss = loss_functin(output, y)
            
        batch_loss += loss.item()

    print("EVAL LOSS", batch_loss / len(test_ids))

    

  0%|          | 0/35 [00:01<?, ?it/s]


RuntimeError: Found dtype Double but expected Float